Python実践データ分析100本ノック（秀和システム）の勉強記録を載せています。<br>
問題の解説、コードを載せています。参考になれば幸いです。<br>
わからない点等ございましたらコメントお願いします。¶

* # ノック61 : 輸送最適化問題を解く

* pulpモジュールは最適化モデルの作成を行う。
* ortoolpyモジュールは目的関数を生成して解く。

* ## 目的関数の形と制約条件（参考）

目的関数$W$は各経路の輸送コストを$c_k$(万円/個)、輸送量を$w_k$(個)として、

$$ W = \sum^k c_k w_k$$

ただし、和は全ての経路に対してとる。

制約条件は、

1. 倉庫の部品の供給量を下回っていること
2. 工場の部品の需要量（最低限の必要量）を満たしていること

の二つである。

In [1]:
# kaggleなら二つのモジュールが入っているのでダウンロードする必要はなし
import numpy as np
import pandas as pd

from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

# データの読み込み
df_tc = pd.read_csv('../input/python100knock-logistic/trans_cost.csv', index_col='工場')
df_demand = pd.read_csv('../input/python100knock-logistic/demand.csv')
df_supply = pd.read_csv('../input/python100knock-logistic/supply.csv')

# 初期設定
np.random.seed(1)
nw = len(df_tc.index)
nf = len(df_tc.columns)
pr = list(product(range(nw), range(nf)))

# 数理モデル作成（目的関数の作成）
# 目的関数は前ノックと同様に、（一個当たりのコスト） * （輸送量）
m1 = model_min()
# m1 : 目的関数 、　v1 : 変化しうる値（これを変化させて最適化を行う
v1 = {(i,j) : LpVariable('v%d_%d'%(i, j), lowBound=0) for i,j in pr}
m1 += lpSum(df_tc.iloc[i, j] * v1[i, j] for i,j in pr)

# 制約条件を与える
for i in range(nw):
    # 供給量に関する制約条件
    m1  += lpSum(v1[i, j] for j in range(nf)) <= df_supply.iloc[0, i]
for j in range(nf):
    # 必要量に関する制約条件
    m1 += lpSum(v1[i, j] for i in range(nw)) >= df_demand.iloc[0, j]
    

m1.solve()

# 総輸送コスト計算
df_tr_sol = df_tc.copy()
total_cost = 0
for k,x in v1.items():
    i,j = k[0], k[1]
    df_tr_sol.iloc[i, j] = value(x)
    total_cost += df_tc.iloc[i, j] * value(x)
    
print(df_tr_sol)
print('総輸送コスト : ' + str(total_cost))




ModuleNotFoundError: No module named 'ortoolpy'

# ノック62 : 最適輸送ルートをネットワークで確認する

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# データ読み込み
df_tr = df_tr_sol.copy()
df_pos = pd.read_csv('../input/python100knock-logistic/trans_route_pos.csv')

# グラフオブジェクトの作成
G = nx.Graph()

# 頂点の設定
for i in range(len(df_pos.columns)):
    G.add_node(df_pos.columns[i])
    
# 辺の設定 & エッジの重みのリスト化
num_pre = 0
edge_weights = []
size = 0.1
for i in range(len(df_pos.columns)):
    for j in range(len(df_pos.columns)):
        if not(i == j):
            # 辺の追加
            G.add_edge(df_pos.columns[i], df_pos.columns[j])
            # エッジの重みの追加
            if num_pre < len(G.edges):
                num_pre = len(G.edges)
                weight = 0
                if (df_pos.columns[i] in df_tr.columns)and(df_pos.columns[j] in df_tr.index):
                    if df_tr_new[df_pos.columns[i]][df_pos.columns[j]]:
                        weight = df_tr[df_pos.columns[i]][df_pos.columns[j]]*size
                elif(df_pos.columns[j] in df_tr.columns)and(df_pos.columns[i] in df_tr.index):
                    if df_tr[df_pos.columns[j]][df_pos.columns[i]]:
                        weight = df_tr[df_pos.columns[j]][df_pos.columns[i]]*size
                edge_weights.append(weight)
            
# 座標の設定

pos = {}
for i in range(len(df_pos.columns)):
    node = df_pos.columns[i]
    pos[node] = (df_pos[node][0],df_pos[node][1])
    
# 描画
nx.draw(G, pos, with_labels=True,font_size=16, node_size = 1000, node_color='k', font_color='w', width=edge_weights)

# 表示
plt.show()


ノック57で立てた「輸送ルートはある程度集約するべき」という仮説が正しかったことが視覚的に明らかになった。

# ノック63 : 最適輸送ルートが制約条件内に収まっているか確認する

In [ ]:
df_demand = pd.read_csv('../input/python100knock-logistic/demand.csv')
df_supply = pd.read_csv('../input/python100knock-logistic/supply.csv')

df_demand.head()

In [ ]:
# 下のコードの参考のためにdf_trの形を出力
df_tr.head()

In [ ]:
# 制約条件計算関数
# 需要側
def condition_demand(df_tr, df_demand):
    flag = np.zeros(len(df_demand.columns))
    for i in range(len(df_demand.columns)):
        # ある工場に輸送されてきた部品の個数を足し上げる（表を縦に足し上げる）
        temp_sum = sum(df_tr[df_demand.columns[i]])
        # 輸送された部品の個数が需要量を上回っていればフラグ立てる
        if (temp_sum >= df_demand.iloc[0, i]):
            flag[i] = 1
            
    return flag
# 供給側
def condition_supply(df_tr, df_supply):
    flag = np.zeros(len(df_supply.columns))
    for i in range(len(df_supply.columns)):
        # ある倉庫から出荷される部品の個数を足し上げる（表を横に足し上げる）
        part_sum_tmp = sum(df_tr.loc[df_supply.columns[i]])
        if part_sum_tmp <= df_supply.iloc[0, i]:
            flag[i] = 1
    return flag

print('需要条件計算結果 : ' + str(condition_demand(df_tr_sol, df_demand)))
print('供給条件計算結果 : ' + str(condition_supply(df_tr_sol, df_supply)))

全ての制約条件が満たされていることが確認できた

今回の問題は、**線形最適化**と呼ばれる問題。比較的短時間で解ける方（らしい）。

# ノック64 : 生産計画に関するデータを読み込む

|No.| ファイル名 | 概要 |
| :-- | :-- | :-- |
| 1 | product_plan_material.csv | 製品の製造に必要な原料の割合 |
| 2 | product_plan_profit.csv | 製品の利益 |
| 3 | product_plan_stock.csv | 原料の在庫 |
| 4 | product_plan.csv | 製品の生産量 |

In [ ]:
# 製品の製造に必要な原料の個数
df_material = pd.read_csv('../input/python100knock-product-plan/product_plan_material.csv', index_col ='製品')
df_material.head()

どうやら今回は原料1と原料2のみで考えるみたい。<br>
原料3も考慮したかったら、下の df_profitの下に原料3の利益を足せば動くはず

In [ ]:
# 製品ごとの利益（単位 : 万円）
df_profit = pd.read_csv('../input/python100knock-product-plan/product_plan_profit.csv', index_col='製品')

# 製品3を追加
# df_profit.loc['製品3'] = 3.0
df_profit.head()



In [ ]:
# 原料の在庫数
df_stock = pd.read_csv('../input/python100knock-product-plan/product_plan_stock.csv', index_col='項目')
df_stock.head()

In [ ]:
# （現在の）製品の生産量
# ここを最適化することで利益を最大化する
df_plan = pd.read_csv('../input/python100knock-product-plan/product_plan.csv', index_col='製品')
df_plan.head()

つまり、現在は製品1のみが生産されている。

# ノック65 : 利益を計算する関数の作成

In [ ]:
# 利益計算関数
def product_plan(df_profit, df_plan):
    profit = 0
    for i in range(len(df_profit.index)):
        for j in range(len(df_plan.columns)):
            # 利益は各製品ごとの利益と製造量の積
            profit += df_profit.iloc[i, j] * df_plan.iloc[i, j]
    return profit

print('総利益 : ' + str(product_plan(df_profit, df_plan)))

今回は製品1の製造のみによる結果。次は製品2も増やす。

# ノック66 : 生産最適化問題を解く

* ###  今回は、先ほど（ノック65）で定式化した目的関数である利益計算関数を最大化することを目的とする。

In [ ]:
from pulp import LpVariable, lpSum, value
from ortoolpy import model_max, addvars, addvals

df = df_material.copy()
inv = df_stock

# 数理モデルの定義（目的関数の作成）
m = model_max()
# v1は製品数と同じ次元数で定義
v1 = {(i):LpVariable('v%d'%(i), lowBound=0) for i in range(len(df_profit))}
# 「変数v1（製品数）」と「製品ごとの利益の和」で目的関数を定義
m += lpSum(df_profit.iloc[i] * v1[i] for i in range(len(df_profit)))

# 制約条件の設定
# それぞれの原料の使用量が在庫を超えないようにする
for i in range(len(df_material.columns)):
    m += lpSum(df_material.iloc[j, i] * v1[j] for j in range(len(df_profit))) <= df_stock.iloc[:, i]
    
# 最適化問題を解く
m.solve()

df_plan_sol = df_plan.copy()
# v1 => [(0, v0), (1, v1)]
for k,x in v1.items():
    df_plan_sol.iloc[k] = value(x)
    

print(df_plan_sol)
print('総利益 : ' + str(value(m.objective)))

# ノック67 : 最適生産計画が制約条件内に収まっているか確認

#### 最適化計算を行った結果を鵜呑みにしてはいけない。<br>→目的関数と制約条件が現実とかけ離れてしまっている可能性がある。

#### 最適化計算を行った結果を「あの手この手で」理解することが重要。<br>今回は「それぞれの原料の使用量」と「在庫を効率よく利用できているか」を確認する。

In [ ]:
# 制約条件計算関数
def condition_stock(df_plan, df_material, df_stock):
    flag = np.zeros(len(df_material.columns))
    for i in range(len(df_material.columns)):
        temp_sum = 0
        for j in range(len(df_material.index)):
            # 各原料ごとに使用数を算出する
            temp_sum = temp_sum + df_material.iloc[j, i] * float(df_plan.iloc[j])
        # 原料の使用数が倉庫の在庫量を下回っていれば1にする（制約条件を満たしている）
        if (temp_sum <= float(df_stock.iloc[0, i])):
            flag[i] = 1
        print(df_material.columns[i] + ' 使用量 : ' + str(temp_sum) + ', 在庫 : ' + str(float(df_stock.iloc[0, i])))
    return flag

print('制約条件計算結果 : ' + str(condition_stock(df_plan_sol, df_material, df_stock)))

原料1だけがちょっと余っているが、他はキチキチに仕えている。よって今回の最適化計画は合理的であると判断できる。

コメント<br>
まあ今回はたまたま原料3の利益がなくてもキチキチに使えるように設定されている気がする。
本当なら原料3の利益を追加して計算し直すべき。

# ノック68 : ロジスティクスネットワーク設計問題を解く

今回は、目的関数として、「輸送コストと製造コストの和」を使用。<br>制約条件として、各焦点での販売数が需要数を上回るようにする。

In [ ]:
製品 = list('AB')
需要地 = list('PQ')
工場 = list('XY')
レーン = (2,2)

# 輸送費表
tbdi = pd.DataFrame(((j,k) for j in 需要地 for k in 工場), columns=['需要地', '工場'])
tbdi['輸送費'] = [1,2,3,1]

tbdi

本の冒頭に示されていた図の通りに輸送費を設定。


In [ ]:
# 需要表
tbde = pd.DataFrame(((j,i) for j in 需要地 for i in 製品), columns=['需要地', '製品'])
tbde['需要'] = [10,10,20,20]

tbde

お店P、お店Qごとに製品の需要を設定。

In [ ]:
# 生産表
tbfa = pd.DataFrame(((k, l, i, 0, np.inf) for k, nl in zip(工場, レーン) for l in range(nl) for i in 製品),
                    columns=['工場', 'レーン', '製品', '下限', '上限'])
tbfa['生産費'] = [1, np.nan, np.nan, 1, 3, np.nan, 5, 3]
tbfa.dropna(inplace=True)
tbfa.loc[4, '上限'] = 10

tbfa

In [ ]:
from ortoolpy import logistics_network

# 輸送ネットワークの最適化を行う
_, tbdi2, _ = logistics_network(tbde, tbdi, tbfa)

# 最適化後の精算表と最適生産量（ValY）を表示
tbfa

In [ ]:
# 最適化後の輸送費表と最適輸送量（ValX）を表示
tbdi2

logistics_networkモジュールがどうやら目的関数の設定から制限の設定まで全部やってくれているみたい。表を渡すだけで最適化が終わっている。

# ノック69 : 最適ネットワークにおける輸送コストとその内訳の計算

In [ ]:
trans_cost = 0
for i in range(len(tbdi2.index)):
    trans_cost += tbdi2['輸送費'].iloc[i] * tbdi2['ValX'].iloc[i]

print('総輸送コスト : ' + str(trans_cost))

（本によると）概ね妥当なコストとなっているらしい

# ノック70 : 最適ネットワークにおける生産コストとその内訳の計算

In [ ]:
product_cost = 0
for i in range(len(tbfa.index)):
    product_cost += tbfa['生産費'].iloc[i] * tbfa['ValY'].iloc[i]

print('総生産コスト : ' + str(product_cost))

概ね妥当らしい

## 6章と7章全体（ロジスティックネットワークの最適化）を通してのコメント

### 正直こんなに簡単に終わる話ではないと思う。目的関数を、制約条件に従って最小化（最大化）するのがネットワークの最適化ということはわかった。<br>ただ、後半のネットワークの最適化は説明不足もあり（中のアルゴリズムの理解不足で）狐につままれるような気分になった。詳細をQiitaにまとめても良いかもしれない。

### 追記<br>
どうやらortoolpyモジュールは作者独自作製のモジュールでブラックボックスな部分が多いらしい。下記のURLの記事が非常に参考になったので載せておく。[「Python実践データ分析100本ノック」実践編②最適化問題をortoolpyを用いずに解く](https://zenn.dev/pipipiz/articles/90323416c2caba)